In [1]:
import tensorflow as tf
import os
import numpy as np
from scipy.spatial import distance

In [2]:
#filename = './stsim_m_fea.txt'
#filename = "./lbp_riu2_comb_fea.txt"
filename = "./mbp_fea.txt"
imagevec_size = 256

In [3]:
with open(filename) as f:
    data = f.readlines()
    gvec_matrix = []
    gidlist = []
    for line in data:
        line = line.split()
        gid = line[0]
        gvec = list(map(float, line[1:]))
        if len(gvec) != imagevec_size:
            print("Image vector error!!! ", len(gvec))
        else:
            gvec_matrix.append(gvec)
            gidlist.append(gid)

In [4]:
len(gvec_matrix[0])

256

In [5]:
len(gidlist)

1181

### Calculate covariance matrix

In [ ]:
X = np.array(gvec_matrix)
D = np.cov(X.T)
VI = np.linalg.inv(D)

## TopK

In [6]:
gidlist = np.array(gidlist)
gnum = len(gvec_matrix)
imagematrix = tf.constant(gvec_matrix)
ginputs = tf.placeholder(tf.int32)
iembedding_batch = tf.reshape(tf.nn.embedding_lookup(imagematrix, ginputs), [1, imagevec_size])
image_vec_tile = tf.tile(iembedding_batch, [gnum, 1])
# euclidean distance
edistance = -tf.norm(image_vec_tile - imagematrix, ord=2, axis=1)
#m_dist = -distance.mahalanobis(image_vec_tile,imagematrix,VI)
topknum = 10
topkscore, topkind = tf.nn.top_k(edistance, topknum)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
output = open("./mbp_top10.txt", 'w+')
for i in range(len(gvec_matrix)):
    tscore, ind = sess.run([topkscore, topkind], feed_dict={ginputs: i})
    line = str(gidlist[i]) + ","
    line1 = ""
    for j in range(topknum):
        line1 += (str(gidlist[ind[j]]) + ",")
    score = ",".join(list(map(str,tscore)))
    line = line + line1 + score + '\n'
    output.write(line)
output.close()